# Introdução

Um grande fator importante de automatização de processos é a manipulação de dados. Podemos automatizar um processo usando por exemplos o proprio VBA mas no nosso caso usaremos o Python. 

Como automatização de processos pode ajudar na organização de dados ? De data frames ? Como podemos tornar o processo automático ? Qual a importancia disso para o mercado ? Qual o impacto de automatização de processos em limpagem e filtragem de dados ?



Crie exemplos que ilustrem a utilidade de se automazir tais processos. 

### Codigo:

O exemplo abaixo mostra um caso real e importante para se entender automatização de processos no qual estamos fazendo um Backup de fotos, todos os dias as no horário já pré-programado. O interessante deste caso é que não precisamos de intervanção humana, basta apenas executar o codigo uma vez e todo dia nesse horário tera feito backup.

In [ ]:

import os
import shutil
import datetime
import schedule
import time

source_dir = "C:/Users/edgar/OneDrive/Imagens/Capturas de tela/rpa_fotos"
destination_dir = "C:/Users/edgar/OneDrive/Imagens/Capturas de tela/Backups"

def copy_folder_to_directory(source, dest):
    today = datetime.date.today()
    dest_dir = os.path.join(dest, str(today))
    
    try:
        shutil.copytree(source, dest_dir)
        print(f"Folder copied to: {dest_dir}")
    except FileExistsError:
        print(f"Folder already exists in: {dest}")


schedule.every().day.at("11:48").do(lambda: copy_folder_to_directory(source_dir, destination_dir))

while True:
    schedule.run_pending()
    time.sleep(60)

Folder already exists in: C:/Users/edgar/OneDrive/Imagens/Capturas de tela/Backups


## Aumentando o Grau de Complexidade



In [ ]:
import os
import shutil
import datetime
import schedule
import time

source_dir = "C:/Users/edgar/OneDrive/Imagens/Capturas de tela/rpa_fotos"
destination_base = "C:/Users/edgar/OneDrive/Imagens/Capturas de tela/Backups"
log_file = "arquivos_copiados.txt"  # Controle global dos arquivos copiados

# Lê o log de arquivos copiados de todos os dias
def carregar_arquivos_copiados():
    if not os.path.exists(log_file):
        return set()
    with open(log_file, "r", encoding="utf-8") as f:
        return set(line.strip() for line in f.readlines())

# Atualiza o log de arquivos copiados
def atualizar_log(arquivo):
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(arquivo + "\n")

def get_today_folder():
    today_str = datetime.date.today().strftime("%d-%m-%Y")
    return os.path.join(destination_base, f"Backups_{today_str}")

def ensure_today_folder_exists():
    today_folder = get_today_folder()
    if not os.path.exists(today_folder):
        os.makedirs(today_folder)
    return today_folder

# Função principal
def copy_new_files():
    today_folder = ensure_today_folder_exists()
    arquivos_copiados = carregar_arquivos_copiados()

    for filename in os.listdir(source_dir):
        source_file = os.path.join(source_dir, filename)
        dest_file = os.path.join(today_folder, filename)

        # Copia só arquivos ainda não registrados no log
        if os.path.isfile(source_file) and filename not in arquivos_copiados:
            try:
                shutil.copy2(source_file, dest_file)
                atualizar_log(filename)
                print(f"Arquivo copiado para {today_folder}: {filename}")
            except Exception as e:
                print(f"Erro ao copiar {filename}: {e}")

# Agendamento a cada 1h
schedule.every(1).minutes.do(copy_new_files)

# Executa a primeira vez imediatamente
copy_new_files()

while True:
    schedule.run_pending()
    time.sleep(60)


Arquivo copiado para C:/Users/edgar/OneDrive/Imagens/Capturas de tela/Backups\Backups_11-06-2025: Captura de tela 2025-06-07 184818.png
Arquivo copiado para C:/Users/edgar/OneDrive/Imagens/Capturas de tela/Backups\Backups_12-06-2025: Captura de tela 2024-09-29 145514.png
Arquivo copiado para C:/Users/edgar/OneDrive/Imagens/Capturas de tela/Backups\Backups_12-06-2025: Captura de tela 2025-06-07 190334.png


## Explicando o Código

O Exemplo anterior, não era necessariamente um processo automatizado. Ele de fato automatizava a copia e cola dos arquivos, assim como o processo de criação da pasta de Backup, mas depois disso o codigo perdia o sentido pois depois da criação da pasta, este não executava mais nenhuma tarefa.

O código acima é bem mais completo nesse sentido. Primeiro que ele cria uma pasta nova assim que cria um dia novo. Segundo que ele detecta de 1 em 1 minuto se houve adicições novas na nossa pasta rpa_fotos e por fim, ele nunca irá repetir arquivos.

Não faz sentido ele copiar e colar todos os arquivos da pasta rpa_fotos sem poder diferenciar aqueles que já sofreram backup. Logo, eu criei funções que permitem meu programa fazer essa diferenciação.
```
def atualizar_log(arquivo):
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(arquivo + "\n")
```

Se o arquivo arquivos_copiados.txt não existir ainda, o Python cria ele automaticamente no modo "a" (append). Cada nome de arquivo copiado é gravado em uma nova linha.


```
def carregar_arquivos_copiados():
    if not os.path.exists(log_file):
        return set()
    with open(log_file, "r", encoding="utf-8") as f:
        return set(line.strip() for line in f.readlines())
```

Essa função Constrói um set com todos os nomes que já foram copiados anteriormente. Assim, o script sabe quais não precisa copiar de novo, mesmo que ainda estejam na pasta rpa_fotos.
